In [96]:
import pandas as pd
import numpy as np

## for language detection
import langdetect 

### Import north american job postings

In [97]:
us_glass_df = pd.read_csv(r'./Glassdoor_WebS/Glassdoor_data_info_us.csv')
us_indeed_df = pd.read_csv(r'./Indeed_WebS/Indeed_data_info_us.csv')
us_link_df = pd.read_csv(r'./LinkedIn_WebS/Linkedin_data_info_us.csv')

In [98]:
us_glass_df.head(2)

,Unnamed: 0,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
0,1,Marketing Data Analyst,LS Networks,"Portland, OR",7/26/2021,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NaN,"Are you a talented, motivated individual who w..."
1,2,"Senior Analyst, Public Health Data Modernization",ASTHO,"Arlington, VA",7/26/2021,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NaN,"Summary: The Senior Analyst, Public Health Dat..."


In [99]:
(us_glass_df.drop('Unnamed: 0',axis=1,inplace=True))
us_glass_df.dropna(how='all',inplace=True)

In [100]:
us_glass_df.head(2)

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
0,Marketing Data Analyst,LS Networks,"Portland, OR",7/26/2021,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NaN,"Are you a talented, motivated individual who w..."
1,"Senior Analyst, Public Health Data Modernization",ASTHO,"Arlington, VA",7/26/2021,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NaN,"Summary: The Senior Analyst, Public Health Dat..."


In [101]:
us_indeed_df.columns

Index(['Unnamed: 0', 'Job Title', 'Company Name', 'Location', 'Date Scraped',
       'Job URL', 'Job Industry', 'Job Type', 'Job Seniority',
       'Job Description'],
      dtype='object')

In [102]:
us_indeed_df.drop('Unnamed: 0',axis=1,inplace=True)

In [103]:
us_indeed_df.head(2)

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
0,\nComplaint Data Analyst,Olympus Corporation of the Americas,"Center Valley, PA 18034",7/25/2021,https://www.indeed.com/rc/clk?jk=7c96832d3bdbc...,NaN,NaN,NaN,Are you looking for a company that cares about...
1,\nData Analyst - Respiratory Therapy Enterprise,Cleveland Clinic,"Cleveland, OH 44101",7/25/2021,https://www.indeed.com/rc/clk?jk=6f046824c8d2e...,NaN,NaN,NaN,Responsibilities:\nPull data from enterprise d...


In [104]:
us_link_df.columns

Index(['Job Title', 'Company Name', 'Location', 'Date Scraped', 'Job URL',
       'Job Industry', 'Job Type', 'Job Seniority', 'Job Description'],
      dtype='object')

In [105]:
us_link_df.head(2)

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
0,Asset Management Analyst,Ten-X,"Richmond, VA",2021-07-26,https://www.linkedin.com/jobs/view/asset-manag...,"Commercial Real Estate, Information Technology...",Employment type\nFull-time,Associate,Asset Management Analyst OVERVIEW CoStar Group...
1,Software Engineer – Biologics,Genedata,"Lexington, MA",2021-07-26,https://www.linkedin.com/jobs/view/software-en...,"Biotechnology, Computer Software, and Informat...",Employment type\nFull-time,Mid-Senior level,Software Engineer – Biologics The biopharmaceu...


## Let the EDA begin

How will we handle it? We have to make a few decisions:
1. Will we find the most required skills for all of the countries altogether? Or will we address each separately?
2. Will we use any of the columns to group our information or analyze the dataset as a whole?
3. How will we work on the long text data? 

1. As we will be deciding beforehand which country i'm moving to, I will address each country required skills separately
2. We will be looking for skills required for each position. It's likely that we will have to standirize the Job titles as different companies call the same positions different names
3. As we dive deeper into the data, we will be exploring different options.

First, we will concatenate all the dfs to be analyze the country data as a whole

In [106]:
len(us_glass_df)

269

In [107]:
len(us_indeed_df)

1352

In [108]:
len(us_link_df)

7156

In [109]:
df_us_concat = pd.concat([us_glass_df,
us_indeed_df,
us_link_df])

In [110]:
len(df_us_concat)

8777

In [111]:
df_us_concat.head(2)

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
0,Marketing Data Analyst,LS Networks,"Portland, OR",7/26/2021,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NaN,"Are you a talented, motivated individual who w..."
1,"Senior Analyst, Public Health Data Modernization",ASTHO,"Arlington, VA",7/26/2021,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NaN,"Summary: The Senior Analyst, Public Health Dat..."


To simplify analysis, we will reset the index

In [112]:
df_us_concat.reset_index(drop=True,inplace=True)

### Language detection

Just to make sure we have reliable information, we will be using a language detection tool

In [113]:
txt = df_us_concat["Job Description"].iloc[0]
print(txt[0:40], " --> ", langdetect.detect(txt))

Are you a talented, motivated individual  -->  en


In [114]:
#df_us_concat['lang'] = df_us_concat.fillna('')['Job Description'].apply(lambda x: langdetect.detect(x) if str(x).strip() != "" else "")
#df_us_concat.head()

In [115]:
#df_us_concat['lang'].value_counts()

#### Checking the job title distribution

In [116]:
round(df_us_concat['Job Title'].value_counts()/len(df_us_concat)*100,2)[0:10]

Software Development Engineer                        5.87
Software Engineer                                    3.59
Software Engineer II                                 2.21
Data Analyst                                         2.03
Business Analyst                                     2.01
Data Engineer                                        1.29
Data Scientist                                       0.93
Software Development Engineer II                     0.75
Volunteer: Flexible Data Support for Brandworkers    0.69
Senior Financial Analyst                             0.60
Name: Job Title, dtype: float64

As we can see, the prevalent job title is Software engineer with a considreable difference. Let's focus on the analysts job offerings.

In [117]:
# Saving the rows that comply with the condition set
df_business_us = df_us_concat[df_us_concat['Job Title'].str.contains('Business').fillna(False)]

In [118]:
df_analyst_us = df_us_concat[df_us_concat['Job Title'].str.contains('Analyst').fillna(False)]

In [119]:
df_data_analytics_us = df_us_concat[df_us_concat['Job Title'].str.contains('Data Analytics').fillna(False)]

In [120]:
df_tableau_us = df_us_concat[df_us_concat['Job Title'].str.contains('Tableau').fillna(False)]

In [121]:
df_bi_us = df_us_concat[df_us_concat['Job Title'].str.contains('Business Intelligence').fillna(False)]

In [122]:
df_risk_us = df_us_concat[df_us_concat['Job Title'].str.contains('Risk').fillna(False)]

In [123]:
df_customer_us = df_us_concat[df_us_concat['Job Title'].str.contains('Customer').fillna(False)]

In [124]:
df_product_us = df_us_concat[df_us_concat['Job Title'].str.contains('Product').fillna(False)]

In [125]:
df_insights_us = df_us_concat[df_us_concat['Job Title'].str.contains('Insights').fillna(False)]

In [126]:
df_full_concat_us = pd.concat([df_business_us,df_analyst_us,df_data_analytics_us,df_tableau_us,df_bi_us,df_risk_us,df_customer_us,df_product_us,df_insights_us])

In [127]:
df_full_concat_us.drop_duplicates(inplace=True)

In [128]:
len(df_full_concat_us)

4359

In [129]:
df_full_concat_us.head()

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
8,"(ROH1477) Wealth Management Chief Data Office,...","JPMorgan Chase Bank, N.A.","Newark, DE",7/26/2021,https://www.glassdoor.com/partner/jobListing.h...,Industry : Finance,Job Type : Full-time,NaN,This role will fulfill a critical role within ...
16,Business Analyst/Data Analyst (on Data Enginee...,Softcrylic,Remote,2021-07-27,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NaN,"Who We Are\nFor 20 years, we have been working..."
18,Junior Business Analyst,First Home Mortgage Corporation,Remote,2021-07-27,https://www.glassdoor.com/partner/jobListing.h...,Industry : Finance,Job Type : Full-time,NaN,First Home Mortgage Corporation has a full-tim...
34,Business Analyst(Scientific Background) - BB40...,"TechData Service Company, LLC","Lawrenceville, NJ",2021-07-27,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NaN,Business Analyst – 40234-1\nBB40234-1\nLawrenc...
51,Business Analyst(Scientific Background) - BB40...,"TechData Service Company, LLC","Lawrenceville, NJ",2021-07-28,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NaN,Business Analyst – 40234-1\nBB40234-1\nLawrenc...


### Text analysis

As we won't be modelling, we will be looking for some commonly asked for skills and we will measure how much they are mentioned in different job postings. For this, we will be using the str.contains method as used above. 

##### Skills that we will be looking for
Below we will list the skills that will be looking for in the descriptions in order to understand which ones are fetched the most:
- Data Visualization (data viz - Visualizing Data - Reporting)
- Power BI (PowerBI)
- Tableus
- Looker
- Qlik
- Google Data Studio (GDS)
- Data Cleaning (Data prep - Data preparation)
- Programming
- MATLAB
- R
- Python
- SAS
- SQL 
- NoSQL
- Machine Learning (ML)
- Microsoft Excel
- ETL
- Azure
- AWS
- Google Cloud Platform (GCP)
- Oracle
- Domain (industry)

In [130]:
skills_list = ['data visualization','data viz','visualizing data','reporting','power bi','powerbi',\
'tableau','looker','qlik','google data studio', 'gds', 'data cleaning', 'data prep','data preparation'\
,'programming', 'matlab', ' r ', 'python', 'sas','sql', 'nosql','machine learning',' ml ','microsoft excel'\
,'excel','etl','azure','aws','google cloud platform','gcp','google','oracle','apache','spark'\
,'dataproc','databricks','airflow','dbt','pipeline','scalding','hadoop','pig']

In [131]:
df_full_concat_us.head()

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
8,"(ROH1477) Wealth Management Chief Data Office,...","JPMorgan Chase Bank, N.A.","Newark, DE",7/26/2021,https://www.glassdoor.com/partner/jobListing.h...,Industry : Finance,Job Type : Full-time,NaN,This role will fulfill a critical role within ...
16,Business Analyst/Data Analyst (on Data Enginee...,Softcrylic,Remote,2021-07-27,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NaN,"Who We Are\nFor 20 years, we have been working..."
18,Junior Business Analyst,First Home Mortgage Corporation,Remote,2021-07-27,https://www.glassdoor.com/partner/jobListing.h...,Industry : Finance,Job Type : Full-time,NaN,First Home Mortgage Corporation has a full-tim...
34,Business Analyst(Scientific Background) - BB40...,"TechData Service Company, LLC","Lawrenceville, NJ",2021-07-27,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NaN,Business Analyst – 40234-1\nBB40234-1\nLawrenc...
51,Business Analyst(Scientific Background) - BB40...,"TechData Service Company, LLC","Lawrenceville, NJ",2021-07-28,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NaN,Business Analyst – 40234-1\nBB40234-1\nLawrenc...


### Looking for the terms on the Job Descriptions

In [132]:
skills_count_dict = {}

In [133]:
for i in skills_list:
    print(i,' was found in ',len(df_full_concat_us[df_full_concat_us['Job Description'].str.lower().str.contains(i).fillna(False)]),' job postings')
    skills_count_dict[i]=len(df_full_concat_us[df_full_concat_us['Job Description'].str.lower().str.contains(i).fillna(False)])

data visualization  was found in  253  job postings
data viz  was found in  0  job postings
visualizing data  was found in  9  job postings
reporting  was found in  1296  job postings
power bi  was found in  320  job postings
powerbi  was found in  112  job postings
tableau  was found in  405  job postings
looker  was found in  64  job postings
qlik  was found in  91  job postings
google data studio  was found in  8  job postings
gds  was found in  1  job postings
data cleaning  was found in  25  job postings
data prep  was found in  35  job postings
data preparation  was found in  35  job postings
programming  was found in  329  job postings
matlab  was found in  12  job postings
 r   was found in  83  job postings
python  was found in  435  job postings
sas  was found in  193  job postings
sql  was found in  954  job postings
nosql  was found in  4  job postings
machine learning  was found in  126  job postings
 ml   was found in  17  job postings
microsoft excel  was found in  112  

In [134]:
# Let's take a look at our dict
skills_count_dict

{'data visualization': 253,
 'data viz': 0,
 'visualizing data': 9,
 'reporting': 1296,
 'power bi': 320,
 'powerbi': 112,
 'tableau': 405,
 'looker': 64,
 'qlik': 91,
 'google data studio': 8,
 'gds': 1,
 'data cleaning': 25,
 'data prep': 35,
 'data preparation': 35,
 'programming': 329,
 'matlab': 12,
 ' r ': 83,
 'python': 435,
 'sas': 193,
 'sql': 954,
 'nosql': 4,
 'machine learning': 126,
 ' ml ': 17,
 'microsoft excel': 112,
 'excel': 1543,
 'etl': 193,
 'azure': 154,
 'aws': 521,
 'google cloud platform': 13,
 'gcp': 10,
 'google': 187,
 'oracle': 152,
 'apache': 3,
 'spark': 61,
 'dataproc': 0,
 'databricks': 5,
 'airflow': 1,
 'dbt': 35,
 'pipeline': 155,
 'scalding': 0,
 'hadoop': 22,
 'pig': 1}

In [135]:
skills_df_us = pd.DataFrame(skills_count_dict.items(),columns=['skill','count'])

In [136]:
skills_df_us.head()

,skill,count
0,data visualization,253
1,data viz,0
2,visualizing data,9
3,reporting,1296
4,power bi,320


In [137]:
skills_df_us['mentions_percent'] = round(skills_df_us['count']/len(df_full_concat_us)*100,2)

In [138]:
skills_df_us=skills_df_us.sort_values('mentions_percent',ascending= False)

In [139]:
skills_df_us

,skill,count,mentions_percent
24,excel,1543,35.40
3,reporting,1296,29.73
19,sql,954,21.89
27,aws,521,11.95
17,python,435,9.98
6,tableau,405,9.29
14,programming,329,7.55
4,power bi,320,7.34
0,data visualization,253,5.80
25,etl,193,4.43


## Exporting the results

In [140]:
#skills_df_us.to_csv(r'C:\Users\Gonzalo\Documents\DH\Contenido\ds_blend_students_2020\Proyectos Propios\By Industry\0 - Jobs\Data Analytics\Results - CSV\results_us.csv')